# This notebook prepares the raw data for plotting.
## First we load the data
## Then we process the data
## Finally we output the processed data as a .csv that can be uploaded to Github. (The raw data is too large for a free account)

<h2> Step 1: Load the data </h2>

<h2> Step 2: Process the data </h2>

<h2> Step 3: Output the processed data </h2>